In [1]:
import rust_machine_learning
import pandas as pd
from syed_ml_lib import *

In [2]:
import os
from PIL import Image
import numpy as np

# Collect images into a dataframe, with their corresponding labels
image_directory = "../data/animals-10/"

# EXPECTED DIRECTORY STRUCTURE
# image_directory
#   classification_1
#       all images of classification_1 in jpg format here
#   classification_2
#       all images of classification_2 in jpg format here

# This is just for the animals, but it's a decent idea to have the labels be the folder names
folders = [folder_name for folder_name in os.listdir(image_directory) if os.path.isdir(image_directory + "/" + folder_name)]
print("Classifications:", folders)

image_data = []

for folder in folders:
    folder_path = image_directory + "/" + folder
    for file in os.listdir(folder_path):

        if len(image_data) > 1:
            break

        image = Image.open(os.path.join(folder_path + "/" + file))
        pixel_data = np.array(image)
        print(folder, file)
        print(pixel_data.shape)
        # print(pixel_data)
        image_data.append(pixel_data)

df = pd.DataFrame({'ImageData': image_data})

df

Classifications: ['pecora', 'cane', 'ragno', 'cavallo', 'gallina', 'scoiattolo', 'farfalla', 'gatto', 'elefante', 'mucca']
pecora OIP-kG4mUcDV0exFgh2Q7owdowAAAA.jpeg
(156, 220, 3)
pecora OIP--S44hiu6-CBwY5MH11WF-QHaEK.jpeg
(169, 300, 3)
pecora OIP-z4Sh4W1KnyzhQcQ0d-i7dgAAAA.jpeg
(225, 300, 3)
pecora OIP-lLgci1j3s-6OeozvPWFbSgHaGt.jpeg
(272, 300, 3)
pecora OIP-H3BYOUMKSkq-HTamw3iK8wHaE8.jpeg
(200, 300, 3)
pecora OIP-sy26HF5k6Dco1-qmpOkq1QHaFj.jpeg
(225, 300, 3)
pecora OIP-QC3WdnrfcGU27VdR0gsOpAAAAA.jpeg
(199, 300, 3)
pecora OIP-EkAjXyiRiQIFpn_adgzXDAHaEK.jpeg
(169, 300, 3)
pecora eb31b8062ff2063ed1584d05fb1d4e9fe777ead218ac104497f5c978a6eab2b0_640.jpg
(427, 640, 3)
pecora OIP-w0tR4ivCs1IxhMfU5vdpRAHaFH.jpeg
(207, 300, 3)
pecora ea35b30a20f2073ed1584d05fb1d4e9fe777ead218ac104497f5c978a6e8b0b1_640.jpg
(418, 640, 3)
pecora OIP-fv_xAJZ1OQdovfRsGUVlHAHaEK.jpeg
(169, 300, 3)
pecora OIP-lLAkD05WLi0pQvykmzYHcgHaFD.jpeg
(205, 300, 3)
pecora OIP-ozG1lrGmUJnKq6Zd4373RAHaFn.jpeg
(228, 300, 3)
pecor

KeyboardInterrupt: 

In [ ]:
to_encode = [] # We won't encode the label column. The output layer will use 0-n (assuming classifications are ordered starting at 0), to determine error.
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
5826,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18030,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5233,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4286,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9132,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9693,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31906,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28052,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Remove unnecessary data or NAN
to_remove = []
encoded = encoded.drop(encoded.columns[to_remove], axis=1)
encoded

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
5826,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18030,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5233,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4286,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9132,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9693,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31906,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28052,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Prepare data for training with input rust functions expect
label_col = 0

# Normalize columns
labels = encoded.iloc[:, label_col]
max_per_col = encoded.abs().max().replace(0,1)
encoded = (encoded / max_per_col)
encoded.iloc[:, label_col] = labels

train_data, test_data = split_train_test(encoded, 0.5, True)

# Remove any encoded data of the same class
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)
train_data_features_list = train_data_features.values.tolist()

test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features_list = test_data_features.values.tolist()

# Data Correlation
# corr = train_data_features.corr()
# fig = px.imshow(corr, aspect="auto", origin='lower')
# fig.show()

# Get labels
train_data_labels = train_data.iloc[:, label_col].values.tolist()
test_data_labels = test_data.iloc[:,label_col].values.tolist()
train_data_features

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
22397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Just showing one of the images as an example
row = 0
img_data = train_data_features.iloc[row].values
classification = train_data_labels[row]
print("Classification for row " + str(row) + " is: " + str(classification))
view_image(img_data, 28, 28, 'gray')

Classification for row 0 is: 0.0


In [ ]:
# Now train the model
hidden_layer_sizes = [10]
num_classifications = 10
learning_rate = 1e-2
num_iterations = 5000
batch_size = 64
trained_model = rust_machine_learning.BasicNeuralNetwork(train_data_features_list, train_data_labels, hidden_layer_sizes, num_classifications, learning_rate, num_iterations, batch_size)

In [ ]:
# Now let us test the model
labelled_results = trained_model.classify(test_data_features_list)

num_correct = 0
tolerance = 0.05
for test_label, result_label in zip(test_data_labels, labelled_results):
  num_correct += (abs(test_label-result_label)/(1.0 if test_label == 0 else test_label) <= tolerance)
print("Percent Correct: ", 100.0 * num_correct / len(test_data_labels))

Percent Correct:  51.26


In [ ]:
# Interactive Test
import random
row = random.randint(0, len(test_data_features))
img_data = test_data_features.iloc[row].values
list_img_data = [test_data_features_list[row]]
prediction = trained_model.classify(list_img_data)[0]
print(prediction)
view_image(img_data, 28, 28, 'gray')

9.0
